In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
from utils import *

In [2]:
df_tennis=pd.read_csv('../data/prepared_data.csv')
df_tennis.head()

,player_id,player_name,player_hand,player_ht,player_age,ace,df,svpt,1stIn,1stWon,...,SvGms,bpSaved,bpFaced,rank,opponent_rank,surface,tourney_level,best_of,target,month
0,100644,Alexander Zverev,R,198,26,10,1,66,47,33,...,10,3,4,7,5,Hard,A,3,1,11
1,100644,Alexander Zverev,R,198,25,5,1,44,27,16,...,8,3,7,15,97,Hard,M,3,0,3
2,100644,Alexander Zverev,R,198,26,5,5,90,66,50,...,15,3,6,19,81,Clay,A,3,1,7
3,100644,Alexander Zverev,R,198,26,3,1,41,33,26,...,7,2,2,19,122,Clay,A,3,1,7
4,100644,Alexander Zverev,R,198,26,7,0,55,42,20,...,8,2,6,10,50,Hard,M,3,0,10


In [3]:
df_tennis.columns

Index(['player_id', 'player_name', 'player_hand', 'player_ht', 'player_age',
       'ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved',
       'bpFaced', 'rank', 'opponent_rank', 'surface', 'tourney_level',
       'best_of', 'target', 'month'],
      dtype='object')

In [4]:
#let's check for null values 
df_tennis.isnull().sum()

player_id        0
player_name      0
player_hand      0
player_ht        0
player_age       0
ace              0
df               0
svpt             0
1stIn            0
1stWon           0
2ndWon           0
SvGms            0
bpSaved          0
bpFaced          0
rank             0
opponent_rank    0
surface          0
tourney_level    0
best_of          0
target           0
month            0
dtype: int64

In [5]:
df_tennis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5498 entries, 0 to 5497
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      5498 non-null   int64 
 1   player_name    5498 non-null   object
 2   player_hand    5498 non-null   object
 3   player_ht      5498 non-null   int64 
 4   player_age     5498 non-null   int64 
 5   ace            5498 non-null   int64 
 6   df             5498 non-null   int64 
 7   svpt           5498 non-null   int64 
 8   1stIn          5498 non-null   int64 
 9   1stWon         5498 non-null   int64 
 10  2ndWon         5498 non-null   int64 
 11  SvGms          5498 non-null   int64 
 12  bpSaved        5498 non-null   int64 
 13  bpFaced        5498 non-null   int64 
 14  rank           5498 non-null   int64 
 15  opponent_rank  5498 non-null   int64 
 16  surface        5498 non-null   object
 17  tourney_level  5498 non-null   object
 18  best_of        5498 non-null

In [6]:
df_tennis.drop('player_id',axis=1,inplace=True)

df_tennis.head()

,player_name,player_hand,player_ht,player_age,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,rank,opponent_rank,surface,tourney_level,best_of,target,month
0,Alexander Zverev,R,198,26,10,1,66,47,33,11,10,3,4,7,5,Hard,A,3,1,11
1,Alexander Zverev,R,198,25,5,1,44,27,16,7,8,3,7,15,97,Hard,M,3,0,3
2,Alexander Zverev,R,198,26,5,5,90,66,50,8,15,3,6,19,81,Clay,A,3,1,7
3,Alexander Zverev,R,198,26,3,1,41,33,26,4,7,2,2,19,122,Clay,A,3,1,7
4,Alexander Zverev,R,198,26,7,0,55,42,20,8,8,2,6,10,50,Hard,M,3,0,10


<p> Good shape now for our model </p>

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
#label encoder for player_hand
le=LabelEncoder()
df_tennis['player_hand_encoded']=le.fit_transform(df_tennis['player_hand'])

#one hot encode
df_tennis=pd.get_dummies(df_tennis, columns=['surface', 'tourney_level'], drop_first=True)

#train test split
X=df_tennis.drop(['target', 'player_name', 'player_hand'], axis=1)

y=df_tennis['target']

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

model=LogisticRegression()

model.fit(X_train, y_train)

y_pred=model.predict(X_test)

accuracy=accuracy_score(y_test, y_pred)
print('Accuracy: ', accuracy)



Accuracy:  0.78


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
threshold=0.5
test_set_probability_of_winning=model.predict_proba(X_test) #predict_proba returns probability of each class
test_set_predictions=(test_set_probability_of_winning[:,1]>threshold).astype(int) #checking if probability of winning is greater than threshold

In [10]:
from sklearn.metrics import  roc_auc_score, classification_report

print("classification_report: \n", classification_report(y_test, test_set_predictions))

#Receiver Operating Characteristic - Area Under the Curve 
roc_auc=roc_auc_score(y_test, test_set_probability_of_winning[:,1])
print('ROC_AUC: ', roc_auc)

classification_report: 
               precision    recall  f1-score   support

           0       0.79      0.76      0.78       551
           1       0.77      0.80      0.78       549

    accuracy                           0.78      1100
   macro avg       0.78      0.78      0.78      1100
weighted avg       0.78      0.78      0.78      1100

ROC_AUC:  0.8547102635050035


<p>A higher ROC-AUC score indicates better discrimination and the model is effective distinguishing between the classes. So the model has good separability <p>

In [11]:
!pip install fuzzywuzzy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
#Using to find the right spelling of a player's name
from fuzzywuzzy import process

name_to_search="Mateo Arnaldi"

best_match=process.extractOne(name_to_search, df_tennis['player_name'])

print(best_match)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


('Matteo Arnaldi', 96, 5211)


In [47]:

# Create a new data point for prediction
player1_dict={
    'player1': 'Novak Djokovic',
    'player1_rank': 1,
    'player1_age': 36,
}
player2_dict={
    'player2': 'Daniil Medvedev',
    'player2_rank': 3,
    'player2_age': 27,
}

match_dict={
    'surface': 'Clay',
    'tourney_level': 'G',
    'best_of':5
}


new_data_point = data_point(player1_dict, player2_dict, match_dict)


# label encoding for player_hand
new_data_point['player_hand_encoded'] = le.transform(new_data_point['player_hand'])

# one hot encoding for surface and tourney_level
new_data_point = pd.get_dummies(new_data_point, columns=['surface', 'tourney_level'], prefix=['surface', 'tourney_level'])

# droping player_hand column
new_data_point = new_data_point.drop(['player_hand'], axis=1)

#if there are any missing columns, add them and fill them with 0
missing_cols = set(X_train.columns) - set(new_data_point.columns)
for col in missing_cols:
    new_data_point[col] = 0

# Ensure the order of column in the test set is in the same order than in train set
new_data_point = new_data_point[X_train.columns]

# Make Prediction
probability_of_winning = model.predict_proba(new_data_point)

# Print the predicted probabilities

print(f"Probability of {player1_dict['player1']} winning: {probability_of_winning[0, 1]:.2%}")
print(f"Probability of {player2_dict['player2']} winning: {probability_of_winning[1, 1]:.2%}")

Probability of Novak Djokovic winning: 73.38%
Probability of Daniil Medvedev winning: 8.29%


![Example Image](./tournament.png)


In [16]:
#let's save the model
import joblib

joblib.dump(model, 'logistic_model.joblib')

['logistic_model.joblib']